In [9]:
import pandas as pd
import glob
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

In [14]:
# Grayson's path
train_folder = "C:\\Users\\grays\\Will-Grayson GitHub Repo\\will-grayson-ML\\nineteen-class\\data\\train\\"
test_folder = "C:\\Users\\grays\\Will-Grayson GitHub Repo\\will-grayson-ML\\nineteen-class\\data\\test\\"

# Will's path (comment out if not in use)
# train_folder = "C:\\Users\\willg\\OneDrive\\CSCI\\summer-2024-work\\will-grayson-ML\\train\\"
# test_folder = "C:\\Users\\willg\\OneDrive\\CSCI\\summer-2024-work\\will-grayson-ML\\test\\"

# Function to load and concatenate CSV files from a list of file paths
def load_and_concat(files, sample_fraction=None):
    df_list = []
    for file in files:
        df = pd.read_csv(file)
        if sample_fraction:
            df = df.sample(frac=sample_fraction, random_state=42)  # Random sampling
        df_list.append(df)
    return pd.concat(df_list, ignore_index=True)

# List all CSV files in the train and test folders
train_files = glob.glob(train_folder + "*.csv")
test_files = glob.glob(test_folder + "*.csv")

# Load and concatenate training and testing data
train_df = load_and_concat(train_files)
test_df = load_and_concat(test_files)


In [15]:
# Combine target labels into a single column for multi-class classification
# Assuming the labels are one-hot encoded
target_columns = ['spoofing', 'benign', 'MQTT_DDoS_Connect_Flood', 'MQTT_DoS_Connect_Flood', 
    'MQTT_DDoS_Publish_Flood', 'MQTT_DoS_Publish_Flood', 'MQTT_Malformed_Data',
    'Recon_OS_Scan', 'Recon_Ping_Sweep', 'Recon_Port_Scan', 'Recon_VulScan',
    'DoS_ICMP', 'DoS_SYN', 'DoS_TCP', 'DoS_UDP',
    'DDoS_ICMP', 'DDoS_SYN', 'DDoS_TCP', 'DDoS_UDP'
]
feature_columns = [col for col in train_df.columns if col not in target_columns]

# Separate features and labels
X_train = train_df[feature_columns]
y_train = train_df[target_columns]
X_test = test_df[feature_columns]
y_test = test_df[target_columns]


In [16]:
# Encode the labels to numerical values
label_encoder = LabelEncoder()
y_train_labels = label_encoder.fit_transform(y_train.idxmax(axis=1))
y_test_labels = label_encoder.transform(y_test.idxmax(axis=1))

# Create a scaler object
scaler = MinMaxScaler()

# Fit the scaler to the training features and transform both training and testing features
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [17]:
# Initialize the individual models
rf = RandomForestClassifier(n_estimators=100, random_state=42)
gb = GradientBoostingClassifier(random_state=42)
svc = SVC(probability=True, random_state=42)

# Initialize the Voting Classifier
voting_clf = VotingClassifier(estimators=[('rf', rf), ('gb', gb), ('svc', svc)], voting='soft')
##voting_clf = VotingClassifier(estimators=[('rf', rf), ('svc', svc)], voting='soft')
# Train the Voting Classifier
voting_clf.fit(X_train_scaled, y_train_labels)

# Make predictions
y_pred_labels = voting_clf.predict(X_test_scaled)

# Evaluate the model
all_labels = list(range(len(label_encoder.classes_)))  # Ensure we have all labels present
print(confusion_matrix(y_test_labels, y_pred_labels, labels=all_labels))
print(classification_report(y_test_labels, y_pred_labels, target_names=label_encoder.classes_, labels=all_labels))

[[3496    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    1    0]
 [   0 1710    0    1    0    0    0    0    0    0    0    0    0    1
     0    6    0    6    0]
 [   0    0 1826    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0]
 [   0    0    0 3620    0    0    0    0    0    0    0    0    0    0
     0    0    0    1    0]
 [   2    0    0    0  979    0    0    0    0    0    0    0    0    0
     0    0    0    3    0]
 [   0    1    0    0    0  984    0    0    0    0    0    0    0    1
     0    0    0    0    0]
 [   0    0    2    0    0    0  819    0    0    0    0    0    0    0
     0    0    0    0    0]
 [   0    0    0 1342    0    0    0   34    0    0    0    0    0    0
     0    0    0    0    0]
 [   0    0    0    0    0    0    0    0  419    0    0    0    0    0
     0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    9    0   74    0    0
     0    0    0    1    0]


c:\Users\grays\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\grays\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\grays\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif